## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-4-turbo"}

In [6]:
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor

current_directory = os.getcwd()


executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir=current_directory,
)


## The task

In [7]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [8]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [9]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [10]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

In [11]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": current_directory,
        "use_docker": False,
    },
)

In [12]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

In [15]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia over the past month, you will need to collect and analyze certain data. Here’s how you can proceed:

### Information Needed
1. **Historical Stock Prices of Nvidia (from 2024-03-23 to 2024-04-23)**: 
   - Daily stock prices including open, high, low, and closing prices.
   - Volume of shares traded each day.

2. **Overall Market Performance Indicators** (optional, but useful for context):
   - Performance of major indices (e.g., S&P 500, NASDAQ) over the same timeframe to compare Nvidia’s performance relative to the market.

3. **Significant Company or Industry Events**:
   - Any major news, product announcements, or financial rep


--------------------------------------------------------------------------------

Next speaker: Executor


>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
                               Close     Volume  Daily % Change
Date                                                           
2024-03-25 00:00:00-04:00  95.001999  552136000             NaN
2024-03-26 00:00:00-04:00  92.560997  513648000       -2.569422
2024-03-27 00:00:00-04:00  90.250000  586067000       -2.496729
2024-03-28 00:00:00-04:00  90.356003  435212000        0.117455
2024-04-01 00:00:00-04:00  90.362999  452441000        0.007743
2024-04-02 00:00:00-04:00  89.452003  433064000       -1.008151
2024-04-03 00:00:00-04:00  88.963997  370067000       -0.545551
2024-04-04 00:00:00-04:00  85.904999  434965000       -3.438467
2024-04-05 00:00:00-04:00  88.008003  399678000        2.448058
2024-04-08 00:00:00-04:00  87.133003  28322000

## Add a speaker selection policy

In [16]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": current_directory,
        "use_docker": False,
    },
)

In [17]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [18]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia over the past month, you will need to collect and analyze certain data. Here’s how you can proceed:

### Information Needed
1. **Historical Stock Prices of Nvidia (from 2024-03-23 to 2024-04-23)**: 
   - Daily stock prices including open, high, low, and closing prices.
   - Volume of shares traded each day.

2. **Overall Market Performance Indicators** (optional, but useful for context):
   - Performance of major indices (e.g., S&P 500, NASDAQ) over the same timeframe to compare Nvidia’s performance relative to the market.

3. **Significant Company or Industry Events**:
   - Any major news, product announcements, or financial rep


--------------------------------------------------------------------------------

Next speaker: Executor


>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
                               Close     Volume  Daily % Change
Date                                                           
2024-03-25 00:00:00-04:00  95.001999  552136000             NaN
2024-03-26 00:00:00-04:00  92.560997  513648000       -2.569422
2024-03-27 00:00:00-04:00  90.250000  586067000       -2.496729
2024-03-28 00:00:00-04:00  90.356003  435212000        0.117455
2024-04-01 00:00:00-04:00  90.362999  452441000        0.007743
2024-04-02 00:00:00-04:00  89.452003  433064000       -1.008151
2024-04-03 00:00:00-04:00  88.963997  370067000       -0.545551
2024-04-04 00:00:00-04:00  85.904999  434965000       -3.438467
2024-04-05 00:00:00-04:00  88.008003  399678000        2.448058
2024-04-08 00:00:00-04:00  87.133003  28322000


--------------------------------------------------------------------------------
